In [29]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, monotonically_increasing_id
from pyspark.sql.types import (StructType,
                               StructField,
                               StringType,
                               IntegerType,
                               DoubleType,
                               FloatType,
                               LongType,
                               TimestampType)

In [2]:
spark = SparkSession.builder.appName("data_spark_on_emr").getOrCreate()

22/05/26 11:05:11 WARN Utils: Your hostname, OneForAll-NickdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.6 instead (on interface en6)
22/05/26 11:05:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 11:05:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/26 11:05:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

In [ ]:
# Stop spark session if I don't need it.
# spark.stop()

In [3]:
# spark session setting configuration
spark.sparkContext.getConf().getAll()

[('spark.sql.parquet.binaryAsString', 'true'),
 ('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1653534314166'),
 ('spark.sql.warehouse.dir',
  'file:/Users/oneforall_nick/workspace/Udacity_capstone_project/aws_emr_steps/spark-warehouse'),
 ('spark.executor.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.host', '172.20.10.6'),
 ('spark.driver.port', '59507'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.name', 'data_spark_on_emr'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1653534312431'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
# READ ALL DATA AND SAVE TO simulation data file, and then analysis those data in spark.
# ****** Read data from local file system(be a s3) to spark DataFrame ******
# All data format will be saved as a parquet file.

# file path: data >> immigration_data
# There are three tables: immigration_table, immigration_personal_table and immigration_label_table
# ***immigration_table, immigration_personal_table and immigration_label_table schema***
""""Table: immigration_main_information schema
pk: cicid -> imm_main_cic_id
1. i94yr  -> imm_year
2. i94mon -> imm_month
3. i94citi&i94res -> imm_citi_res -> imm_cntyl
4. i94visa -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
5. i94port -> imm_port
6. arrdate -> imm_arrival_date:
7. i94mode -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
8. i94addr -> imm_address
    ex: 'AL'='ALABAMA'
9. airline -> imm_airline
10 fltno -> imm_flight_no
    schema: StringType()
"""


"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> imm_per_cic_id
    schema: StringType()
1. biryear -> imm_person_birth_year
    schema: IntegerType()
2. gender -> imm_person_gender
    schema: StringType()
3. visatype -> imm_person_visa_type
    schema: StringType()
"""

"""Table: immigration_label schema


"""


In [15]:
# file path: data >> news_data
"""Table: news_article schema
pk: cord_uid -> news_cord_uid
1. source_x -> news_source
    schema: StringType()
2. title -> news_title
    schema: StringType()
3. license -> news_licence
    schema: StringType()
4. abstract -> news_abstract
    schema: StringType()
5. publish_time -> news_publish_time (fk)
    schema: TimestampType()
6. authors -> news_authors
    schema: StringType()
7. url -> news_url
    schema: StringType()
"""
data_news = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/news_data/metadata.csv"

df_news = pd.read_csv(data_news, header=0, delimiter=',')
df_news.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [12]:
# file path: data >> usCitiesDemographics_data
data_us_cities_demographics = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/usCitiesDemographics_data/usCitiesDemo.csv"

df_data_us_cities_demographics = pd.read_csv(data_us_cities_demographics, header=0, delimiter=";")

df_data_us_cities_demographics


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,Stockton,California,32.5,150976.0,154674.0,305650,12822.0,79583.0,3.16,CA,American Indian and Alaska Native,19834
2887,Southfield,Michigan,41.6,31369.0,41808.0,73177,4035.0,4011.0,2.27,MI,American Indian and Alaska Native,983
2888,Indianapolis,Indiana,34.1,410615.0,437808.0,848423,42186.0,72456.0,2.53,IN,White,553665
2889,Somerville,Massachusetts,31.0,41028.0,39306.0,80334,2103.0,22292.0,2.43,MA,American Indian and Alaska Native,374


In [30]:
# Create a us-cities data dimension table
"""Table: us_cities_demographics schema
pk: generated -> cidemo_id
    schema: IntegerType()
1. City -> cidemo_city
    schema: StringType()
2. State -> cidemo_state
    schema: StringType()
3. Median Age -> cidemo_median_age
    schema: FloatType()
4. Total Population -> cidemo_total_population
    schema: IntegerType()
5. State Code -> cidemo_state_code (fk)
    schema: StringType()
6. Count -> cidemo_count
    schema: IntegerType()
"""

# TODO -> Must be defined a function that generated each table schema:
new_data_schema = StructType([\
    StructField(name="cidemo_city", dataType=StringType(), nullable=True),
    StructField(name="cidemo_state", dataType=StringType(), nullable=True),
    StructField(name="cidemo_median_age", dataType=FloatType(), nullable=True),
    StructField(name="cidemo_total_population", dataType=IntegerType(), nullable=True),
    StructField(name="cidemo_state_code", dataType=StringType(), nullable=True),
    StructField(name="cidemo_count", dataType=IntegerType(), nullable=True)
])

# Using pyspark to read csv file
pdf_news = spark.read.options(delimiter=';').csv(data_us_cities_demographics, schema=new_data_schema)
# pdf_news.printSchema()
"""
root
 |-- cidemo_city: string (nullable = true)
 |-- cidemo_state: string (nullable = true)
 |-- cidemo_median_age: float (nullable = true)
 |-- cidemo_total_population: integer (nullable = true)
 |-- cidemo_state_code: string (nullable = true)
 |-- cidemo_count: integer (nullable = true)
"""
# Auto-generated series of id
pdf_news.withColumn("cidemo_id", monotonically_increasing_id()).show(5)

+----------------+-------------+-----------------+-----------------------+-----------------+------------+---------+
|     cidemo_city| cidemo_state|cidemo_median_age|cidemo_total_population|cidemo_state_code|cidemo_count|cidemo_id|
+----------------+-------------+-----------------+-----------------------+-----------------+------------+---------+
|            City|        State|             null|                   null|Female Population|        null|        0|
|   Silver Spring|     Maryland|             33.8|                  40601|            41862|       82463|        1|
|          Quincy|Massachusetts|             41.0|                  44129|            49500|       93629|        2|
|          Hoover|      Alabama|             38.5|                  38040|            46799|       84839|        3|
|Rancho Cucamonga|   California|             34.5|                  88127|            87105|      175232|        4|
+----------------+-------------+-----------------+----------------------

In [4]:
data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_apr16_sub.sas7bdat"

from sas7bdat import SAS7BDAT

# TODO: chunksize??Iteraters??
read = SAS7BDAT(data)

read.header

Header:
	col_count_p1: 28
	col_count_p2: 0
	column_count: 28
	compression: None
	creator: None
	creator_proc: None
	date_created: 2018-05-31 13:19:46.085000
	date_modified: 2018-05-31 13:19:46.085000
	endianess: little
	file_type: DATA
	filename: immigration_apr16_sub.sas7bdat
	header_length: 65536
	lcp: 8
	lcs: 0
	mix_page_row_count: 398
	name: I94_APR16_SUB
	os_name: 
	os_type: 
	page_count: 7201
	page_length: 65536
	platform: windows
	row_count: 3096313
	row_length: 152
	sas_release: 9.0401M4
	server_type: X64_ES08R2
	u64: False

Contents of dataset "I94_APR16_SUB":
Num Name        Type   Length Format Label                                                                                
--- ----------- ------ ------ ------ -------------------------------------------------------------------------------------
  1 cicid       number      8        ID created to merge recoded ri94port variable                                        
  2 i94yr       number      8        Variable created t

In [5]:
# Get data column attributes
for i in read.columns:
    print("col_id ", i.col_id)
    print("  name",  i.name.decode(encoding ='utf-8'))
    print("  label", i.label.decode(encoding ='utf-8'))
    print("  format", i.format)
    print("  type", i.type)
    print("  length", i.length)

col_id  0
  name cicid
  label ID created to merge recoded ri94port variable
  format 
  type number
  length 8
col_id  1
  name i94yr
  label Variable created to capture the year
  format 
  type number
  length 8
col_id  2
  name i94mon
  label Variable created to capture the month
  format 
  type number
  length 8
col_id  3
  name i94cit
  label Country of Citizenship
  format 
  type number
  length 8
col_id  4
  name i94res
  label Country of Residence - i94cit if missing and all collapsed countries
  format 
  type number
  length 8
col_id  5
  name i94port
  label Port of Entry with pre-clearance recoded and all collpsed ports
  format 
  type string
  length 3
col_id  6
  name arrdate
  label Actual admission date - number
  format 
  type number
  length 8
col_id  7
  name i94mode
  label Method of transportation used to arrive in U.S.
  format 
  type number
  length 8
col_id  8
  name i94addr
  label State of intended address in U.S. used for the Summary & Analysis Tables
 

In [ ]:
# In dive data to read
# for r in read:
#     print

In [7]:
df = read.to_data_frame()
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4454822.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1950.0,10222016,F,,2D,9.438122e+10,00402,B2
1,4454823.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1952.0,10222016,F,,2D,9.437935e+10,00402,B2
2,4454824.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1953.0,10222016,M,,OJ,9.430155e+10,00270,B2
3,4454825.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1959.0,10222016,M,,2D,9.437960e+10,00402,B2
4,4454826.0,2016.0,4.0,603.0,603.0,NYC,20567.0,1.0,NY,NaN,...,,,1961.0,10222016,F,,OJ,9.430146e+10,00270,B2
